In [40]:
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from typing import List, Dict, Set, Any, Optional
import sys

class ToughTimetableCSP:
    def __init__(self):
        # More constrained problem definition
        self.subjects = {
            'Math': {'teacher': 'Teacher A', 'type': 'core'},
            'Physics': {'teacher': 'Teacher B', 'type': 'science'},
            'Chemistry': {'teacher': 'Teacher C', 'type': 'science'},
            'Biology': {'teacher': 'Teacher D', 'type': 'science'},
            'PE': {'teacher': 'Teacher E', 'type': 'practical'},
            'CS': {'teacher': 'Teacher A', 'type': 'lab'},  # Same teacher as Math!
            'English': {'teacher': 'Teacher G', 'type': 'core'},
            'History': {'teacher': 'Teacher H', 'type': 'humanities'}
        }
        
        self.time_slots = {
            1: '9:00-10:00',
            2: '10:00-11:00', 
            3: '11:00-12:00',
            4: '1:00-2:00'
        }
        
        self.setup_constraints()
    
    def setup_constraints(self):
        """Setup multiple tough constraints that will require backtracking"""
        self.constraints = []
        
        # Constraint 1: Teacher A teaches both Math and CS - they must be in different slots
        self.constraints.append({
            'type': 'all_different',
            'variables': ['Math', 'CS'],
            'description': 'Teacher A teaches both Math and CS (different slots)'
        })
        
        # Constraint 2: Science subjects (Physics, Chemistry, Biology) must be in different slots
        self.constraints.append({
            'type': 'all_different',
            'variables': ['Physics', 'Chemistry', 'Biology'],
            'description': 'All science subjects must be in different slots'
        })
        
        # Constraint 3: Core subjects (Math, English) must be in different slots
        self.constraints.append({
            'type': 'all_different', 
            'variables': ['Math', 'English'],
            'description': 'Core subjects must be in different slots'
        })
        
        # Constraint 4: Lab subject (CS) must be in slot 3 or 4 (afternoon)
        self.constraints.append({
            'type': 'restricted_slots',
            'variable': 'CS',
            'allowed_slots': {3, 4},
            'description': 'CS must be in afternoon slot (3 or 4)'
        })
        
        # Constraint 5: Practical subject (PE) must be in slot 1 or 4
        self.constraints.append({
            'type': 'restricted_slots',
            'variable': 'PE',
            'allowed_slots': {1, 4},
            'description': 'PE must be in first or last slot (1 or 4)'
        })
        
        # Constraint 6: Math cannot be in slot 1
        self.constraints.append({
            'type': 'restricted_slots',
            'variable': 'Math',
            'allowed_slots': {2, 3, 4},
            'description': 'Math cannot be in first slot'
        })
        
        # Constraint 7: English must be in slot 1 or 2
        self.constraints.append({
            'type': 'restricted_slots',
            'variable': 'English',
            'allowed_slots': {1, 2},
            'description': 'English must be in morning slot (1 or 2)'
        })
        
        # Constraint 8: No teacher should teach two subjects at the same time
        teacher_subjects = defaultdict(list)
        for subject, info in self.subjects.items():
            teacher_subjects[info['teacher']].append(subject)
        
        for teacher, subjects_list in teacher_subjects.items():
            if len(subjects_list) > 1:
                self.constraints.append({
                    'type': 'all_different',
                    'variables': subjects_list,
                    'description': f'Teacher {teacher} cannot teach multiple subjects simultaneously'
                })

class CSPState:
    def __init__(self, variables: List[str], domains: Dict[str, Set[int]]):
        self.variables = variables
        self.domains = domains
        self.assignment: Dict[str, int] = {}
        self.unassigned = set(variables)
        
    def is_complete(self) -> bool:
        return len(self.assignment) == len(self.variables)
    
    def copy(self):
        new_state = CSPState(self.variables, {k: v.copy() for k, v in self.domains.items()})
        new_state.assignment = self.assignment.copy()
        new_state.unassigned = self.unassigned.copy()
        return new_state

class CSPSolver:
    def __init__(self, timetable: ToughTimetableCSP):
        self.timetable = timetable
        self.variables = list(timetable.subjects.keys())
        self.domains = {var: set(timetable.time_slots.keys()) for var in self.variables}
        self.stats = {
            'nodes_explored': 0,
            'backtracks': 0,
            'start_time': 0,
            'solve_time': 0
        }
        
    def is_consistent(self, variable: str, value: int, assignment: Dict[str, int]) -> bool:
        """Check if assigning value to variable violates any constraints"""
        temp_assignment = assignment.copy()
        temp_assignment[variable] = value
        
        for constraint in self.timetable.constraints:
            if not self._check_constraint(constraint, temp_assignment):
                return False
        return True
    
    def _check_constraint(self, constraint: Dict, assignment: Dict[str, int]) -> bool:
        """Check a specific constraint"""
        constraint_type = constraint['type']
        
        if constraint_type == 'all_different':
            vars_in_assignment = [v for v in constraint['variables'] if v in assignment]
            values = [assignment[v] for v in vars_in_assignment]
            return len(values) == len(set(values))
        
        elif constraint_type == 'restricted_slots':
            if constraint['variable'] in assignment:
                return assignment[constraint['variable']] in constraint['allowed_slots']
            return True
        
        return True

class BacktrackingSolver(CSPSolver):
    def solve(self, use_heuristics: bool = True) -> Optional[Dict[str, int]]:
        """Solve using backtracking with optional heuristics"""
        self.stats = {'nodes_explored': 0, 'backtracks': 0, 'start_time': time.time()}
        initial_state = CSPState(self.variables, self.domains)
        solution = self._backtrack(initial_state, use_heuristics)
        self.stats['solve_time'] = time.time() - self.stats['start_time']
        return solution
    
    def _backtrack(self, state: CSPState, use_heuristics: bool) -> Optional[Dict[str, int]]:
        """Recursive backtracking algorithm"""
        self.stats['nodes_explored'] += 1
        
        if state.is_complete():
            return state.assignment
        
        # Select variable using heuristics or simple ordering
        if use_heuristics:
            variable = self._select_variable_mrv(state)
        else:
            variable = next(iter(state.unassigned))
        
        # Order values using heuristics
        if use_heuristics:
            values = self._order_values_lcv(variable, state)
        else:
            values = list(state.domains[variable])
        
        for value in values:
            if self.is_consistent(variable, value, state.assignment):
                # Make assignment
                state.assignment[variable] = value
                state.unassigned.remove(variable)
                
                # Recursive call
                result = self._backtrack(state, use_heuristics)
                if result is not None:
                    return result
                
                # Backtrack
                del state.assignment[variable]
                state.unassigned.add(variable)
                self.stats['backtracks'] += 1
        
        return None
    
    def _select_variable_mrv(self, state: CSPState) -> str:
        """Minimum Remaining Values heuristic"""
        return min(state.unassigned, key=lambda var: len(state.domains[var]))
    
    def _order_values_lcv(self, variable: str, state: CSPState) -> List[int]:
        """Least Constraining Value heuristic"""
        def count_conflicts(value: int) -> int:
            conflicts = 0
            temp_assignment = state.assignment.copy()
            temp_assignment[variable] = value
            
            for other_var in state.unassigned:
                if other_var != variable:
                    for other_value in state.domains[other_var]:
                        if not self.is_consistent(other_var, other_value, temp_assignment):
                            conflicts += 1
            return conflicts
        
        return sorted(state.domains[variable], key=count_conflicts)

class ForwardCheckingSolver(CSPSolver):
    def solve(self, use_heuristics: bool = True) -> Optional[Dict[str, int]]:
        """Solve using backtracking with forward checking"""
        self.stats = {'nodes_explored': 0, 'backtracks': 0, 'start_time': time.time()}
        initial_state = CSPState(self.variables, self.domains)
        solution = self._fc_backtrack(initial_state, use_heuristics)
        self.stats['solve_time'] = time.time() - self.stats['start_time']
        return solution
    
    def _fc_backtrack(self, state: CSPState, use_heuristics: bool) -> Optional[Dict[str, int]]:
        """Backtracking with forward checking"""
        self.stats['nodes_explored'] += 1
        
        if state.is_complete():
            return state.assignment
        
        # Select variable
        if use_heuristics:
            variable = self._select_variable_mrv(state)
        else:
            variable = next(iter(state.unassigned))
        
        # Order values
        if use_heuristics:
            values = self._order_values_lcv(variable, state)
        else:
            values = list(state.domains[variable])
        
        for value in values:
            if self.is_consistent(variable, value, state.assignment):
                # Create new state for forward checking
                new_state = state.copy()
                new_state.assignment[variable] = value
                new_state.unassigned.remove(variable)
                
                # Perform forward checking
                if self._forward_check(variable, value, new_state):
                    result = self._fc_backtrack(new_state, use_heuristics)
                    if result is not None:
                        return result
                
                self.stats['backtracks'] += 1
        
        return None
    
    def _forward_check(self, variable: str, value: int, state: CSPState) -> bool:
        """Perform forward checking and return False if domain wipeout occurs"""
        for other_var in state.unassigned:
            if other_var != variable:
                # Remove inconsistent values
                original_domain = state.domains[other_var].copy()
                state.domains[other_var] = {
                    val for val in original_domain 
                    if self.is_consistent(other_var, val, state.assignment)
                }
                
                # Check for domain wipeout
                if len(state.domains[other_var]) == 0:
                    return False
        return True
    
    def _select_variable_mrv(self, state: CSPState) -> str:
        """Minimum Remaining Values heuristic"""
        return min(state.unassigned, key=lambda var: len(state.domains[var]))
    
    def _order_values_lcv(self, variable: str, state: CSPState) -> List[int]:
        """Least Constraining Value heuristic"""
        def count_conflicts(value: int) -> int:
            conflicts = 0
            temp_assignment = state.assignment.copy()
            temp_assignment[variable] = value
            
            for other_var in state.unassigned:
                if other_var != variable:
                    original_domain_size = len(state.domains[other_var])
                    new_domain_size = len([
                        val for val in state.domains[other_var] 
                        if self.is_consistent(other_var, val, temp_assignment)
                    ])
                    conflicts += (original_domain_size - new_domain_size)
            return conflicts
        
        return sorted(state.domains[variable], key=count_conflicts)

class PerformanceComparator:
    def __init__(self):
        self.results = []
    
    def run_comparison(self, num_trials: int = 30):
        """Run comprehensive performance comparison with single-line progress"""
        print(f"Running performance comparison with {num_trials} trials...")
        print("Progress: [", end="")
        
        success_count = 0
        for trial in range(num_trials):
            # Update progress bar
            progress = (trial + 1) / num_trials
            bars = int(50 * progress)
            spaces = 50 - bars
            sys.stdout.write(f"\rProgress: [{'=' * bars}{' ' * spaces}] {trial + 1}/{num_trials}")
            sys.stdout.flush()
            
            timetable = ToughTimetableCSP()
            
            # Test all method combinations
            methods = [
                ('Backtracking (No Heuristics)', BacktrackingSolver(timetable), False),
                ('Backtracking + Heuristics', BacktrackingSolver(timetable), True),
                ('Forward Checking (No Heuristics)', ForwardCheckingSolver(timetable), False),
                ('Forward Checking + Heuristics', ForwardCheckingSolver(timetable), True)
            ]
            
            for method_name, solver, use_heuristics in methods:
                solution = solver.solve(use_heuristics)
                
                if solution:
                    success_count += 1
                
                self.results.append({
                    'trial': trial,
                    'method': method_name,
                    'solution_found': solution is not None,
                    'solve_time_ms': solver.stats['solve_time'] * 1000,
                    'nodes_explored': solver.stats['nodes_explored'],
                    'backtracks': solver.stats['backtracks'],
                    'has_heuristics': use_heuristics,
                    'has_forward_checking': 'Forward' in method_name
                })
        
        print(f"] COMPLETED! ({success_count//4}/{num_trials} problems solvable)")
        return pd.DataFrame(self.results)
    
    def analyze_results(self, df: pd.DataFrame):
        """Analyze and display results"""
        print("\n" + "="*70)
        print("PERFORMANCE COMPARISON RESULTS")
        print("="*70)
        
        # Calculate statistics
        stats = df.groupby('method').agg({
            'solution_found': ['mean', 'count'],
            'solve_time_ms': ['mean', 'std', 'min', 'max'],
            'nodes_explored': ['mean', 'std'],
            'backtracks': ['mean', 'std']
        }).round(4)
        
        # Format results
        summary = pd.DataFrame({
            'Success Rate (%)': stats[('solution_found', 'mean')] * 100,
            'Avg Time (ms)': stats[('solve_time_ms', 'mean')],
            'Std Time (ms)': stats[('solve_time_ms', 'std')],
            'Avg Nodes': stats[('nodes_explored', 'mean')],
            'Avg Backtracks': stats[('backtracks', 'mean')]
        })
        
        print("\nPerformance Summary:")
        print(summary)
        
        # Create visualizations
        self._create_visualizations(df, summary)
        
        return summary
    
    def _create_visualizations(self, df: pd.DataFrame, summary: pd.DataFrame):
        """Create comprehensive visualizations"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('CSP Solver Performance Comparison - Tough Problem', fontsize=16, fontweight='bold')
        
        methods = summary.index
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
        
        # Success Rate
        bars1 = ax1.bar(methods, summary['Success Rate (%)'], color=colors, alpha=0.7)
        ax1.set_ylabel('Success Rate (%)')
        ax1.set_title('Solution Success Rate')
        ax1.tick_params(axis='x', rotation=45)
        ax1.set_ylim(0, 110)
        for bar, rate in zip(bars1, summary['Success Rate (%)']):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                    f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
        
        # Computation Time
        bars2 = ax2.bar(methods, summary['Avg Time (ms)'], color=colors, alpha=0.7)
        ax2.set_ylabel('Time (milliseconds)')
        ax2.set_title('Average Computation Time')
        ax2.tick_params(axis='x', rotation=45)
        for bar, time_val in zip(bars2, summary['Avg Time (ms)']):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    f'{time_val:.2f}ms', ha='center', va='bottom')
        
        # Nodes Explored
        bars3 = ax3.bar(methods, summary['Avg Nodes'], color=colors, alpha=0.7)
        ax3.set_ylabel('Nodes Explored')
        ax3.set_title('Search Space Exploration')
        ax3.tick_params(axis='x', rotation=45)
        for bar, nodes in zip(bars3, summary['Avg Nodes']):
            ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                    f'{nodes:.0f}', ha='center', va='bottom')
        
        # Backtracks
        bars4 = ax4.bar(methods, summary['Avg Backtracks'], color=colors, alpha=0.7)
        ax4.set_ylabel('Backtrack Operations')
        ax4.set_title('Backtracking Efficiency')
        ax4.tick_params(axis='x', rotation=45)
        for bar, backtracks in zip(bars4, summary['Avg Backtracks']):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    f'{backtracks:.1f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()

def print_final_timetable():
    """Print a clean final timetable allocation table"""
    print("\n" + "="*70)
    print("FINAL TIMETABLE ALLOCATION - TOUGH PROBLEM")
    print("="*70)
    
    # Create and solve the problem
    timetable = ToughTimetableCSP()
    best_solver = ForwardCheckingSolver(timetable)
    solution = best_solver.solve(use_heuristics=True)
    
    if not solution:
        print("❌ No valid timetable could be generated for this tough problem!")
        return
    
    # Create schedule table
    schedule = defaultdict(list)
    for subject, slot in solution.items():
        schedule[slot].append({
            'subject': subject,
            'teacher': timetable.subjects[subject]['teacher'],
            'type': timetable.subjects[subject]['type']
        })
    
    print(f"\n{'Time Slot':<10} {'Time':<12} {'Subject':<12} {'Teacher':<12} {'Type':<12}")
    print("-" * 65)
    
    for slot in sorted(schedule.keys()):
        time_range = timetable.time_slots[slot]
        classes = schedule[slot]
        
        for i, class_info in enumerate(classes):
            if i == 0:
                print(f"{slot:<10} {time_range:<12} {class_info['subject']:<12} {class_info['teacher']:<12} {class_info['type']:<12}")
            else:
                print(f"{'':<10} {'':<12} {class_info['subject']:<12} {class_info['teacher']:<12} {class_info['type']:<12}")
    
    # Print teacher assignments verification
    print("\n" + "="*50)
    print("TEACHER ASSIGNMENT VERIFICATION")
    print("="*50)
    
    teacher_assignments = defaultdict(list)
    for subject, slot in solution.items():
        teacher = timetable.subjects[subject]['teacher']
        teacher_assignments[teacher].append((subject, slot))
    
    for teacher, assignments in teacher_assignments.items():
        if len(assignments) > 1:
            slots = [slot for _, slot in assignments]
            if len(slots) == len(set(slots)):
                print(f"✅ Teacher {teacher}: {', '.join([f'{subj}(Slot {slot})' for subj, slot in assignments])}")
            else:
                print(f"❌ Teacher {teacher}: CONFLICT - {', '.join([f'{subj}(Slot {slot})' for subj, slot in assignments])}")
        else:
            subj, slot = assignments[0]
            print(f"✅ Teacher {teacher}: {subj}(Slot {slot})")

def demonstrate_optimal_solution():
    """Demonstrate the optimal solution with clean output"""
    print("TOUGH TIMETABLE CSP PROBLEM")
    print("="*60)
    
    timetable = ToughTimetableCSP()
    
    print("\nPROBLEM SPECIFICATION:")
    print(f"• Subjects: {len(timetable.subjects)} subjects")
    print(f"• Time Slots: {len(timetable.time_slots)} slots")
    print(f"• Teachers: {len(set(info['teacher'] for info in timetable.subjects.values()))} teachers")
    print(f"• Key Challenge: Teacher A teaches both Math and CS!")
    print(f"• Constraints: {len(timetable.constraints)} tough constraints")
    
    # Find optimal solution using best method
    print("\nFinding optimal timetable solution...")
    best_solver = ForwardCheckingSolver(timetable)
    solution = best_solver.solve(use_heuristics=True)
    
    if solution:
        print("✅ Optimal timetable found for tough problem!")
        print_final_timetable()
        
        print(f"\nSolution Performance:")
        print(f"• Method: Forward Checking with Heuristics (MRV + LCV)")
        print(f"• Solve Time: {best_solver.stats['solve_time']*1000:.2f} ms")
        print(f"• Nodes Explored: {best_solver.stats['nodes_explored']}")
        print(f"• Backtracks: {best_solver.stats['backtracks']}")
    else:
        print("❌ No valid timetable solution found for this tough problem!")

# Run the complete analysis
if __name__ == "__main__":
    # Demonstrate optimal solution
    demonstrate_optimal_solution()
    
    # Print final conclusions
    print("\n" + "="*60)
    print("FINAL CONCLUSIONS - PROBLEM")
    print("="*60)
    
    best_method = summary.loc[summary['Avg Time (ms)'].idxmin()]
    most_efficient = summary.loc[summary['Avg Nodes'].idxmin()]
    most_reliable = summary.loc[summary['Success Rate (%)'].idxmax()]
    
    print("🏆 PERFORMANCE RANKINGS:")
    print(f"1. Fastest: {summary['Avg Time (ms)'].idxmin()} ({best_method['Avg Time (ms)']:.2f} ms)")
    print(f"2. Most Efficient: {summary['Avg Nodes'].idxmin()} ({most_efficient['Avg Nodes']:.0f} nodes)")
    print(f"3. Most Reliable: {summary['Success Rate (%)'].idxmax()} ({most_reliable['Success Rate (%)']:.1f}%)")

TOUGH TIMETABLE CSP PROBLEM

PROBLEM SPECIFICATION:
• Subjects: 8 subjects
• Time Slots: 4 slots
• Teachers: 7 teachers
• Key Challenge: Teacher A teaches both Math and CS!
• Constraints: 8 tough constraints

Finding optimal timetable solution...
✅ Optimal timetable found for tough problem!

FINAL TIMETABLE ALLOCATION - TOUGH PROBLEM

Time Slot  Time         Subject      Teacher      Type        
-----------------------------------------------------------------
1          9:00-10:00   PE           Teacher E    practical   
                        English      Teacher G    core        
                        History      Teacher H    humanities  
                        Physics      Teacher B    science     
2          10:00-11:00  Math         Teacher A    core        
                        Chemistry    Teacher C    science     
3          11:00-12:00  CS           Teacher A    lab         
                        Biology      Teacher D    science     

TEACHER ASSIGNMENT VERIFICATI